## Exploratory Data Analysis

In [1]:
import pandas as pd

In [2]:
products = pd.read_csv("data/products.csv")
sales = pd.read_csv("data/sales.csv")

In [3]:
products.head()

,sku,name,current_price,cost_price,stock
0,A123,Item A,649.99,500,150
1,B456,Item B,699.00,550,15
2,C789,Item C,999.00,500,250


In [4]:
sales.head()

,sku,quantity_sold
0,A123,10
1,B456,35
2,C789,0


In [6]:
products.describe()

,current_price,cost_price,stock
count,3.000000,3.000000,3.000000
mean,782.663333,516.666667,138.333333
std,188.948829,28.867513,117.933597
min,649.990000,500.000000,15.000000
25%,674.495000,500.000000,82.500000
50%,699.000000,500.000000,150.000000
75%,849.000000,525.000000,200.000000
max,999.000000,550.000000,250.000000


In [17]:
sales.describe()

,quantity_sold
count,3.000000
mean,15.000000
std,18.027756
min,0.000000
25%,5.000000
50%,10.000000
75%,22.500000
max,35.000000


## Data Aggregation

In [8]:
sales_summary = sales.groupby('sku')['quantity_sold'].sum()

In [9]:
sales_summary


sku
A123    10
B456    35
C789     0
Name: quantity_sold, dtype: int64

In [10]:
data = pd.merge(products, sales, on='sku')

In [11]:
data

,sku,name,current_price,cost_price,stock,quantity_sold
0,A123,Item A,649.99,500,150,10
1,B456,Item B,699.00,550,15,35
2,C789,Item C,999.00,500,250,0


## Pricing Logic


In [12]:
def apply_pricing_rules(row):
    price = row['current_price']
    cost = row['cost_price']
    stock = row['stock']
    sold = row['quantity_sold']

    new_price = price

    if stock < 20 and sold > 30:
        new_price = price * 1.15

    elif stock > 200 and sold == 0:
        new_price = price * 0.7

    elif stock > 100 and sold  < 20:
        new_price = price * 0.90

    min_price = cost * 1.2
    if new_price < min_price:
        new_price = min_price

    return round(new_price, 2)

In [14]:
data['new_price'] = data.apply(apply_pricing_rules, axis=1)

In [15]:
data

,sku,name,current_price,cost_price,stock,quantity_sold,new_price
0,A123,Item A,649.99,500,150,10,600.00
1,B456,Item B,699.00,550,15,35,803.85
2,C789,Item C,999.00,500,250,0,699.30


## Data Splitting for better output

In [22]:
output = data[['sku', 'current_price', 'new_price']].copy()

In [26]:
output

,sku,old_price,new_price
0,A123,649.99,600.00
1,B456,699.00,803.85
2,C789,999.00,699.30


In [28]:
output.rename(columns={'current_price': 'old_price'}, inplace=True)
output['old_price'] = output['old_price'].apply(lambda x: f"{x:.2f}$")
output['new_price'] = output['new_price'].apply(lambda x: f"{x:.2f}$")

In [29]:
output

,sku,old_price,new_price
0,A123,649.99$,600.00$
1,B456,699.00$,803.85$
2,C789,999.00$,699.30$
